In [ ]:
# File: large-five-twist.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

# make an app
app = App.create("large-five-twist")

# create a large cylinder mesh with specified radius and length
half_length = 2.0 / 0.75
V, F = app.mesh.cylinder(r=1, min_x=-half_length, max_x=half_length, n=1181)

# add to the asset and name it "cylinder"
app.asset.add.tri("cylinder", V, F)

# create a scene
scene = app.scene.create()
objs = []

# arrange 5 cylinders in a circular pattern with larger radius
n, r = 5, 1.75
for i in range(5):
    t = 2 * i / n * np.pi
    x, y = r * np.cos(t), r * np.sin(t)
    objs.append(scene.add("cylinder").at(0, x, y))

# configure material properties and animations for each cylinder
for obj in objs:

    # set bending stiffness, contact gap, Young's modulus, Poisson's ratio and density
    (
        obj.param.set("bend", 500.0)
        .set("contact-gap", 1.75e-3)
        .set("young-mod", 1e4)
        .set("poiss-rat", 0.25)
        .set("density", 3.5)
    )

    # define animation parameters for large twisting motion
    angular_vel, move_delta, scale, t_end, t_wait, s_wait = (
        180,
        1,
        0.25,
        34,
        500 / 60,
        16,
    )
    x, y, z = obj.position

    # pin and animate left end - spin, scale, and move
    left_pin = obj.pin(obj.grab([-1, 0, 0]))
    left_pin.spin(
        axis=[1, 0, 0],
        angular_velocity=angular_vel,
        t_end=t_wait,
    ).scale(
        scale,
        0.0,
        s_wait,
        center=[-x - half_length, -y, -z],
    ).move_by([move_delta, 0, 0], 0.0, t_end)

    # pin and animate right end - spin in opposite direction, scale, and move
    right_pin = obj.pin(obj.grab([1, 0, 0]))
    right_pin.spin(
        axis=[-1, 0, 0],
        angular_velocity=angular_vel,
        t_end=t_wait,
    ).scale(
        scale, 0.0, s_wait, center=[-x + half_length, -y, -z]
    ).move_by([-move_delta, 0, 0], 0.0, t_end)

    # add additional spinning motion
    angular_vel, t_end = 180, 150

    left_pin.spin(
        center=[-x, -y, -z],
        axis=[1, 0, 0],
        angular_velocity=angular_vel,
        t_start=t_wait,
        t_end=t_end,
    )

    right_pin.spin(
        center=[-x, -y, -z],
        axis=[-1, 0, 0],
        angular_velocity=angular_vel,
        t_start=t_wait,
        t_end=t_end,
    )

# compile the scene and report stats
scene = scene.build().report()

# set preview options
opts = {"wireframe": False}

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session params
(
    session.param.set("auto-save", 10)
    .set("dt", 1 / 60)
    .set("frames", 2400)
    .set("gravity", 0.0)
    .set("csrmat-max-nnz", 90000000)
    .set("keep-verts", 100)
)

# build this session
session = session.build()

In [ ]:
# start the simulation (this example takes a long time)
session.start()

In [ ]:
# this example takes a long time...
# in case you shutdown the server (or kernel) and still want to restart
# from where you have (auto) saved, do this. Do not call cells above.

from frontend import App  # noqa

# recover the session from auto-saved state
session = App.recover("large-five-twist")

# resume if not currently running
if not App.busy():
    session.resume()

# preview the current state
session.preview({"wireframe": False})

# stream the logs
session.stream()